# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [4]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [5]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [6]:
# Your code here

# Scale X_train and X_val using StandardScaler
scaler = StandardScaler()

# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
#fit the scaler on the training data and transform it
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
#Convert scaled features into dataframes
X_train_scaled_df = pd.DataFrame(X_train_scaled,columns = X_train.columns)
X_val_scaled_df = pd.DataFrame(X_val_scaled,columns = X_val.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled,columns = X_test.columns)
#display output
print(X_train_scaled_df.head())
print(X_val_scaled_df.head())
print(X_test_scaled_df.head())

    LotArea  OverallQual  OverallCond  TotalBsmtSF  1stFlrSF  2ndFlrSF  \
0 -0.114710    -0.099842    -0.509252    -0.639316 -0.804789  1.261552   
1 -0.176719     0.632038    -0.509252     0.838208  0.641608 -0.808132   
2 -0.246336    -0.831723     1.304613    -0.012560 -0.329000 -0.808132   
3 -0.378617    -0.831723     1.304613    -0.339045 -0.609036 -0.808132   
4 -0.010898    -1.563603     1.304613    -2.531499 -1.315922  0.550523   

   GrLivArea  TotRmsAbvGrd  GarageArea  Fireplaces  
0   0.499114      0.250689    0.327629   -0.994820  
1  -0.247249     -0.365525    0.079146   -0.994820  
2  -0.944766     -0.981739   -1.105931   -0.994820  
3  -1.146010     -0.981739   -1.134602    0.588023  
4  -0.481708      0.250689   -2.281450   -0.994820  
    LotArea  OverallQual  OverallCond  TotalBsmtSF  1stFlrSF  2ndFlrSF  \
0 -0.491264    -0.099842     0.397681    -0.248487 -0.598161 -0.808132   
1 -0.619291     0.632038    -0.509252    -0.205591 -0.549222  0.849426   
2  0.240165    

In [7]:
# Your code here
from sklearn.metrics import r2_score
# Create a LinearRegression model and fit it on scaled training data
linreg = LinearRegression()
#fit the model on the scaled training data
linreg.fit(X_train_scaled_df,y_train)
#predict on the training set
y_train_pred = linreg.predict(X_train_scaled_df)
# Calculate a baseline r-squared score on training data
r2_train = r2_score(y_train,y_train_pred)
print(f"Training R2_score: {r2_train:.4f}")


Training R2_score: 0.7868


## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [8]:
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd

# Step 1: Generate All Possible Combinations of Columns
column_pairs = list(combinations(X_train_scaled_df.columns, 2))

# Store the results in a list
interaction_results = []

# Step 2: Add Interaction Terms and Evaluate R^2 Score
for pair in column_pairs:
    # Create the interaction term
    X_train_interaction = X_train_scaled_df.copy()
    interaction_term = X_train_scaled_df[pair[0]] * X_train_scaled_df[pair[1]]
    interaction_name = f"{pair[0]}_x_{pair[1]}"
    
    # Add the interaction term to the DataFrame
    X_train_interaction[interaction_name] = interaction_term
    
    # Fit the model and calculate R^2 score
    linreg = LinearRegression()
    linreg.fit(X_train_interaction, y_train)
    y_train_pred = linreg.predict(X_train_interaction)
    r2_score_value = r2_score(y_train, y_train_pred)
    
    # Store the pair and its R^2 score
    interaction_results.append((pair, r2_score_value))

# Step 3: Sort the Results by R^2 Score
interaction_results.sort(key=lambda x: x[1], reverse=True)

# Step 4: Print the Top 7 Interactions
top_7_interactions = interaction_results[:7]
print("Top 7 Interactions that Increase the R^2 Score the Most:")
for idx, result in enumerate(top_7_interactions, 1):
    print(f"{idx}. Interaction: {result[0]}, R^2 Score: {result[1]:.4f}")


Top 7 Interactions that Increase the R^2 Score the Most:
1. Interaction: ('OverallQual', 'GarageArea'), R^2 Score: 0.8172
2. Interaction: ('OverallQual', 'Fireplaces'), R^2 Score: 0.8135
3. Interaction: ('OverallQual', 'TotRmsAbvGrd'), R^2 Score: 0.8104
4. Interaction: ('OverallQual', 'GrLivArea'), R^2 Score: 0.8100
5. Interaction: ('OverallQual', 'TotalBsmtSF'), R^2 Score: 0.8100
6. Interaction: ('GrLivArea', 'Fireplaces'), R^2 Score: 0.8080
7. Interaction: ('OverallQual', '1stFlrSF'), R^2 Score: 0.8049


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [9]:
# Assuming top_7_interactions is the list of top 7 interactions obtained from the previous step
# and X_train_scaled_df and X_val_scaled_df are your scaled DataFrames for training and validation sets

# Add the top 7 interactions to X_train and X_val
for pair, _ in top_7_interactions:
    # Create the interaction term
    interaction_name = f"{pair[0]}_{pair[1]}"
    
    # Add interaction term to X_train
    X_train_scaled_df[interaction_name] = X_train_scaled_df[pair[0]] * X_train_scaled_df[pair[1]]
    
    # Add interaction term to X_val
    X_val_scaled_df[interaction_name] = X_val_scaled_df[pair[0]] * X_val_scaled_df[pair[1]]

# Display the first few rows to check the new columns
print("X_train with Interaction Terms:")
print(X_train_scaled_df.head())

print("\nX_val with Interaction Terms:")
print(X_val_scaled_df.head())


X_train with Interaction Terms:
    LotArea  OverallQual  OverallCond  TotalBsmtSF  1stFlrSF  2ndFlrSF  \
0 -0.114710    -0.099842    -0.509252    -0.639316 -0.804789  1.261552   
1 -0.176719     0.632038    -0.509252     0.838208  0.641608 -0.808132   
2 -0.246336    -0.831723     1.304613    -0.012560 -0.329000 -0.808132   
3 -0.378617    -0.831723     1.304613    -0.339045 -0.609036 -0.808132   
4 -0.010898    -1.563603     1.304613    -2.531499 -1.315922  0.550523   

   GrLivArea  TotRmsAbvGrd  GarageArea  Fireplaces  OverallQual_GarageArea  \
0   0.499114      0.250689    0.327629   -0.994820               -0.032711   
1  -0.247249     -0.365525    0.079146   -0.994820                0.050023   
2  -0.944766     -0.981739   -1.105931   -0.994820                0.919828   
3  -1.146010     -0.981739   -1.134602    0.588023                0.943674   
4  -0.481708      0.250689   -2.281450   -0.994820                3.567282   

   OverallQual_Fireplaces  OverallQual_TotRmsAbvGrd  O

## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [10]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd

# Initialize an empty list to store the results
polynomial_results = []

# Step 1: Generate and Evaluate Polynomial Terms for Each Column
for col in X_train_scaled_df.columns:
    for degree in [2, 3, 4]:
        # Create polynomial features of the specified degree for the column
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        X_poly_train = poly.fit_transform(X_train_scaled_df[[col]])
        X_poly_val = poly.transform(X_val_scaled_df[[col]])

        # We only want the pure polynomial term (e.g., col^2, col^3, col^4)
        poly_col_name = f"{col}^{degree}"
        
        # Add the polynomial term to the DataFrame
        X_train_poly = X_train_scaled_df.copy()
        X_train_poly[poly_col_name] = X_poly_train[:, -1]
        
        # Fit the model with the polynomial term
        linreg = LinearRegression()
        linreg.fit(X_train_poly, y_train)
        y_train_pred = linreg.predict(X_train_poly)
        
        # Calculate the R^2 score
        r2_train = r2_score(y_train, y_train_pred)
        
        # Store the result as a tuple: (column name, degree, R^2 score)
        polynomial_results.append((col, degree, r2_train))

# Step 2: Sort the Results by R^2 Score
polynomial_results.sort(key=lambda x: x[2], reverse=True)

# Step 3: Print the Top Polynomial Terms
print("Top Polynomial Terms that Increase the R^2 Score the Most:")
for col, degree, r2 in polynomial_results[:7]:  # Print the top 7 results
    print(f"Column: {col}, Degree: {degree}, R^2 Score: {r2:.4f}")


Top Polynomial Terms that Increase the R^2 Score the Most:
Column: OverallQual_1stFlrSF, Degree: 4, R^2 Score: 0.8774
Column: 1stFlrSF, Degree: 4, R^2 Score: 0.8763
Column: OverallQual_1stFlrSF, Degree: 3, R^2 Score: 0.8732
Column: 1stFlrSF, Degree: 3, R^2 Score: 0.8648
Column: OverallQual_1stFlrSF, Degree: 2, R^2 Score: 0.8613
Column: OverallQual_TotalBsmtSF, Degree: 4, R^2 Score: 0.8538
Column: TotalBsmtSF, Degree: 4, R^2 Score: 0.8529


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [11]:
# Dictionary to store the best polynomial for each column
best_polynomials = {}

# Populate the dictionary with the best polynomial term for each column
for col, degree, r2 in polynomial_results:
    if col not in best_polynomials:
        best_polynomials[col] = (degree, r2)

# Now add these best polynomials to the DataFrames
for col, (degree, _) in best_polynomials.items():
    # Create polynomial features of the specified degree for the column
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly_train = poly.fit_transform(X_train_scaled_df[[col]])
    X_poly_val = poly.transform(X_val_scaled_df[[col]])
    
    # Add the highest degree term to the DataFrames
    poly_col_name = f"{col}^{degree}"
    X_train_scaled_df[poly_col_name] = X_poly_train[:, -1]
    X_val_scaled_df[poly_col_name] = X_poly_val[:, -1]

# Display the updated DataFrames with the new polynomial terms
print("X_train with Polynomial Terms:")
print(X_train_scaled_df.head())

print("\nX_val with Polynomial Terms:")
print(X_val_scaled_df.head())


X_train with Polynomial Terms:
    LotArea  OverallQual  OverallCond  TotalBsmtSF  1stFlrSF  2ndFlrSF  \
0 -0.114710    -0.099842    -0.509252    -0.639316 -0.804789  1.261552   
1 -0.176719     0.632038    -0.509252     0.838208  0.641608 -0.808132   
2 -0.246336    -0.831723     1.304613    -0.012560 -0.329000 -0.808132   
3 -0.378617    -0.831723     1.304613    -0.339045 -0.609036 -0.808132   
4 -0.010898    -1.563603     1.304613    -2.531499 -1.315922  0.550523   

   GrLivArea  TotRmsAbvGrd  GarageArea  Fireplaces  ...  \
0   0.499114      0.250689    0.327629   -0.994820  ...   
1  -0.247249     -0.365525    0.079146   -0.994820  ...   
2  -0.944766     -0.981739   -1.105931   -0.994820  ...   
3  -1.146010     -0.981739   -1.134602    0.588023  ...   
4  -0.481708      0.250689   -2.281450   -0.994820  ...   

   OverallQual_Fireplaces^4  2ndFlrSF^4  OverallCond^2  \
0                  0.000097    2.532911       0.259338   
1                  0.156297    0.426509       0.25933

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [ ]:
# Your code here

## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [ ]:
# Your code here

It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [ ]:
# Your code here


Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [ ]:
# Your code here


Compare the results. Which features would you choose to use?

In [ ]:
# Your written answer here

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [ ]:
# Your code here


### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [ ]:
# Your code here


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.